In [27]:
from src.data_loader import load_data
from src.pipeline import create_pipeline
from src.Preprocessing import Preprocessing
from src.CleoraFacade import CleoraFacade
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from pathlib import Path

In [42]:
import pandas as pd
headers = ['id']
for i in range(42):
    headers.append(i)
df = pd.read_csv("data/features.csv", names=headers, header=0)
print(df)

          id     0     1     2     3     4     5     6     7     8  ...  32  \
0          0  1574  3773  3571  2672  2478  2534  3129  3077  1171  ...   0   
1          1  1193   376    73   290  3129  1852  3077  1171  1022  ...   0   
2          2  1574  3773   925  1728  2815  2963  3077   364  1171  ...   0   
3          3  3964  3773  4003   928  1852  3077   364  1022  3763  ...   0   
4          4  1929  3773  1793  3511  1290  3129  3077   364  1171  ...   0   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...  ..   
37695  37695  1574  3773    73  1995  3554  1233  1789   345  3129  ...   0   
37696  37696  1929  3773  1663  1404   508   819  1852  3077   364  ...   0   
37697  37697  3433  3773  3104  1245  3129  1852  3077   364  1171  ...   0   
37698  37698  3730  3773  1695  2092  2954  1852  3077   364  1171  ...   0   
37699  37699  3433   509  1663  3437   676  3129  1852  3077  3763  ...   0   

       33  34  35  36  37  38  39  40  41  
0      

In [29]:
preprocessing = Preprocessing()
preprocessing.make_preprocessed_edges_file()
cleora = CleoraFacade()
cleora.run_cleora(Path.cwd() / "data" / "preprocessed_edges.txt")

(30160, 3)
          id       name  ml_target
4773    4773      Nsima          0
36287  36287  jaycech3n          0
24241  24241  davendw49          1
27463  27463   mccordgh          0
6815    6815    oalders          0


In [48]:
train = preprocessing.train
test = preprocessing.test
print(train.head())

          id       name  ml_target
4773    4773      Nsima          0
36287  36287  jaycech3n          0
24241  24241  davendw49          1
27463  27463   mccordgh          0
6815    6815    oalders          0


In [49]:
embeddings_path = Path.cwd() / "data" / "embeddings" / "emb__cluster_id__node.out"
embeddings, dimension = cleora.load_embeddings(embeddings_path)

In [50]:
train = train.merge(embeddings, left_on='id', right_on='node', how='left').drop(columns=['node'])
test = test.merge(embeddings, left_on='id', right_on='node', how='left').drop(columns=['node'])
print(train.head())

      id       name  ml_target  \
0   4773      Nsima          0   
1  36287  jaycech3n          0   
2  24241  davendw49          1   
3  27463   mccordgh          0   
4   6815    oalders          0   

                                           embedding  
0  [-0.05318168, -0.05147112, -0.04538219, -0.007...  
1  [0.04182476, 0.016493518, -0.08613632, -0.0226...  
2  [8.01578e-05, -0.009422336, -0.09046693, 0.080...  
3  [-0.01578629, -0.0067136083, 0.057113085, -0.0...  
4  [0.06407978, -0.07241562, 0.11838246, 0.111570...  


In [51]:
train = train.merge(df, on='id')
test = test.merge(df, on='id')
print(train.head())

      id       name  ml_target  \
0   4773      Nsima          0   
1  36287  jaycech3n          0   
2  24241  davendw49          1   
3  27463   mccordgh          0   
4   6815    oalders          0   

                                           embedding     0     1     2     3  \
0  [-0.05318168, -0.05147112, -0.04538219, -0.007...  3730  1330   277  1542   
1  [0.04182476, 0.016493518, -0.08613632, -0.0226...  1929  1330  1793   101   
2  [8.01578e-05, -0.009422336, -0.09046693, 0.080...  3730  1033  1793  2771   
3  [-0.01578629, -0.0067136083, 0.057113085, -0.0...  3730   509  3571  1559   
4  [0.06407978, -0.07241562, 0.11838246, 0.111570...  2048  2325  3214  3129   

      4     5  ...  32  33  34  35  36  37  38  39  40  41  
0  3129  3077  ...   0   0   0   0   0   0   0   0   0   0  
1  2548  1290  ...   0   0   0   0   0   0   0   0   0   0  
2  2947  3129  ...   0   0   0   0   0   0   0   0   0   0  
3  3596  3129  ...   0   0   0   0   0   0   0   0   0   0  
4  1852  

In [55]:
X_test = test.drop(columns=["id", "name", "ml_target"])
X_train = train.drop(columns=["id", "name", "ml_target"])
y_train = train['ml_target']
y_test = test['ml_target']
X_train.head()

,embedding,0,1,2,3,4,5,6,7,8,...,32,33,34,35,36,37,38,39,40,41
0,"[-0.05318168, -0.05147112, -0.04538219, -0.007...",3730,1330,277,1542,3129,3077,364,1022,3763,...,0,0,0,0,0,0,0,0,0,0
1,"[0.04182476, 0.016493518, -0.08613632, -0.0226...",1929,1330,1793,101,2548,1290,3077,364,1171,...,0,0,0,0,0,0,0,0,0,0
2,"[8.01578e-05, -0.009422336, -0.09046693, 0.080...",3730,1033,1793,2771,2947,3129,3077,1171,1022,...,0,0,0,0,0,0,0,0,0,0
3,"[-0.01578629, -0.0067136083, 0.057113085, -0.0...",3730,509,3571,1559,3596,3129,364,1022,3763,...,0,0,0,0,0,0,0,0,0,0
4,"[0.06407978, -0.07241562, 0.11838246, 0.111570...",2048,2325,3214,3129,1852,3077,364,1171,1022,...,0,0,0,0,0,0,0,0,0,0


In [74]:
X_test = []
for i in test.values:
    l = i[3].tolist()
    for x in range(4, len(i)):
        l.append(i[x])
    X_test.append(l)
print(X_test[0])

X_train = []
for i in train.values:
    l = i[3].tolist()
    for x in range(4, len(i)):
        l.append(i[x])
    X_train.append(l)
print(X_train[0])

[-0.08606248, -0.09733334, -0.06357559, -0.08451723, 0.04718374, 0.04506947, -0.1413375, -0.06452729, 0.015114757, -0.11094865, 0.15918387, -0.082086734, 0.12615938, 0.16015774, 0.029101333, 0.0001339671, 0.033882875, -0.021637939, -0.075255014, 0.15640664, 0.058416825, -0.063626975, -0.078586504, -0.08335538, 0.11556307, 0.11392255, -0.0838909, -0.06494134, 0.05126182, -0.052901994, -0.112708226, 0.04113988, 0.090040855, 0.0067846114, -0.06494589, 0.20049052, 0.0050847502, -0.022951836, -0.014135835, -0.104296505, 0.072631106, -0.05304778, -0.1847334, -0.02944122, -0.04254049, -0.0081584565, 0.043075453, -0.15172626, -0.01682658, 0.055153426, -0.06772295, -0.078567125, -0.04974827, -0.088238135, 0.03746905, 0.026892772, 0.0111452555, -0.050254263, 0.024482658, -0.10366168, 0.051508453, 0.016157757, 0.05218183, 0.1343271, -0.060469825, 0.043996327, -0.02197111, -0.1339002, -0.010219006, -0.03190067, 0.11626609, 0.0066494597, 0.033399533, 0.10287354, -0.014606367, 0.10214354, 0.12724727

In [75]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


In [77]:
models = {
    "KNeighbors": KNeighborsClassifier(n_neighbors=7), 
    "DecisionTree": DecisionTreeClassifier() 
}

for model_name, model in models.items():
        print(f"\nTraining pipeline with {model_name}...")
        pipeline = create_pipeline(model)
        
        pipeline.fit(X_train_scaled, y_train)
        y_pred = pipeline.predict(X_test_scaled)
        
        # evaluate model performance
        print(f"\n{model_name} Model Evaluation:")
        print(f"Accuracy: {pipeline.score(X_test_scaled, y_test):.4f}")
        print(classification_report(y_test, y_pred))


Training pipeline with KNeighbors...
[Pipeline] . (step 1 of 2) Processing cosine_similarity, total= 1.5min
[Pipeline] ............... (step 2 of 2) Processing knn, total=   1.6s

KNeighbors Model Evaluation:
Accuracy: 0.7564
              precision    recall  f1-score   support

           0       0.77      0.97      0.86      5620
           1       0.60      0.13      0.22      1920

    accuracy                           0.76      7540
   macro avg       0.68      0.55      0.54      7540
weighted avg       0.72      0.76      0.69      7540


Training pipeline with DecisionTree...
[Pipeline] ........ (step 1 of 1) Processing classifier, total=  11.0s

DecisionTree Model Evaluation:
Accuracy: 0.7422
              precision    recall  f1-score   support

           0       0.83      0.82      0.83      5620
           1       0.49      0.52      0.51      1920

    accuracy                           0.74      7540
   macro avg       0.66      0.67      0.67      7540
weighted avg  